In [25]:
!!pip install --user pgeocode
!!pip install --user plotly==5.3.1


['Requirement already satisfied: plotly==5.3.1 in /home/21132844/.local/lib/python3.8/site-packages (5.3.1)',
 'Requirement already satisfied: tenacity>=6.2.0 in /home/21132844/.local/lib/python3.8/site-packages (from plotly==5.3.1) (8.0.1)',
 'Requirement already satisfied: six in /opt/jupyterhub/anaconda/lib/python3.8/site-packages (from plotly==5.3.1) (1.16.0)']

In [26]:
import pgeocode
import plotly.express as px
import pandas as pd

In [27]:
''' Options for console output '''
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 200)

nomi = pgeocode.Nominatim('NL')


In [38]:
schools_df = pd.read_csv('../Data-Files/Postcodes_Groningen.csv', usecols=['Adres'])
dh = pd.read_csv('../Data-Files/Postcodes_DenHaag_Rotterdam.csv', usecols=['Adres'])

newtryschoo = schools_df
schools_df = schools_df['Adres'].str.split(' ', expand=True)
schools_df = schools_df.drop(len(schools_df.columns)-1, axis=1)
schools_df = schools_df.rename(columns={0: 'postcode_number', 1: 'postcode_string'})

dh = dh['Adres'].str.split(',', expand=True)
dh = dh.drop(0, axis=1)
newtry = dh
dh = dh[1].str.split(' ', expand=True)
dh = dh.rename(columns={1: 'postcode_number', 2: 'postcode_string'})
dh = dh.drop([0,3,4], axis=1)



result = pd.concat([schools_df, dh])
result['type'] = 'school'
result['opacity'] = 1
print('res \n',result)
print(result.info())

res 
    postcode_number postcode_string    type  opacity
0             9977              PP  school        1
1             9472              PA  school        1
2             9713              VA  school        1
3             9761              HJ  school        1
4             9761              TM  school        1
5             9731              CB  school        1
6             9751              RL  school        1
7             9753              HN  school        1
8             9722              JV  school        1
9             9723              EM  school        1
10            9736              CT  school        1
0             3191              GA  school        1
1             3082              VW  school        1
2             2562              SK  school        1
3             2546              SK  school        1
4             2313              AB  school        1
5             2545              NV  school        1
6             2571              VD  school        1
7     

In [42]:
students_df = pd.read_csv('../Output-Files/t0_with_zipcodes.csv', usecols=['Zipcode-number'])
students_df = students_df.dropna()
students_df['Zipcode-number'] = students_df['Zipcode-number'].astype(int)
students_df['postcode_number'] = students_df['Zipcode-number'].astype(str)
students_df['type'] = 'student'
#students_df['new_color'] = students_df.groupby('zipcode_numbers').count()
print(students_df.groupby('Zipcode-number').count())
#students_df = students_df.drop('zipcode_numbers', axis=1)
print(students_df.info())
print(students_df.head())

students_df['latitude'] = students_df['postcode_number'].apply(lambda x: nomi.query_postal_code(x)[9]) 
students_df['longitude'] = students_df['postcode_number'].apply(lambda x: nomi.query_postal_code(x)[10])
students_df['opacity'] = 0.1
print(students_df.head())
print(students_df.isna().sum())

                postcode_number  type
Zipcode-number                       
2264                         34    34
2266                         33    33
2271                         33    33
2313                          9     9
2516                         11    11
2526                        121   121
2545                         45    45
2546                         21    21
2548                         16    16
2562                          8     8
2571                         26    26
3079                         26    26
3082                         40    40
3191                         42    42
9472                         35    35
9713                         13    13
9722                         89    89
9723                         11    11
9751                         66    66
9753                         23    23
9761                         92    92
9977                         37    37
<class 'pandas.core.frame.DataFrame'>
Int64Index: 831 entries, 413 to 1270
Data columns 

In [43]:
result['latitude'] = result['postcode_number'].apply(lambda x: nomi.query_postal_code(x)[9])
result['longitude'] = result['postcode_number'].apply(lambda x: nomi.query_postal_code(x)[10])

schools_and_students_df = pd.concat([result, students_df])
print(schools_and_students_df)

     postcode_number postcode_string     type  opacity  latitude  longitude  \
0               9977              PP   school      1.0   53.3878     6.3895   
1               9472              PA   school      1.0   53.0776     6.6959   
2               9713              VA   school      1.0   53.2253     6.5813   
3               9761              HJ   school      1.0   53.1367     6.5633   
4               9761              TM   school      1.0   53.1367     6.5633   
5               9731              CB   school      1.0   53.2347     6.5905   
6               9751              RL   school      1.0   53.1738     6.6109   
7               9753              HN   school      1.0   53.1722     6.6261   
8               9722              JV   school      1.0   53.1976     6.5866   
9               9723              EM   school      1.0   53.2108     6.5990   
10              9736              CT   school      1.0   53.2494     6.6007   
0               3191              GA   school      1

In [54]:

fig = px.scatter_mapbox(schools_and_students_df, lat="latitude", lon="longitude",  color='type', opacity=schools_and_students_df['opacity'], zoom=6, title="Location of Schools and Students", height=500, )
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(
    title={
        'text': "Location of Schools and Students",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
 
fig.show()


In [50]:
MQ_Category = pd.read_csv('../Output-Files/t0_with_zipcodes.csv', usecols=['MQ category', 'Zipcode-number'])
MQ_Category = MQ_Category.dropna()
print(MQ_Category.head())
print(MQ_Category.info())
MQ_Category['Zipcode-number'] = MQ_Category['Zipcode-number'].astype(int)
MQ_Category['Zipcode-number'] = MQ_Category['Zipcode-number'].astype(str)

MQ_Category['latitude'] = MQ_Category['Zipcode-number'].apply(lambda x: nomi.query_postal_code(x)[9])
MQ_Category['longitude'] = MQ_Category['Zipcode-number'].apply(lambda x: nomi.query_postal_code(x)[10])

     MQ category  Zipcode-number
413          3.0          3191.0
414          1.0          3191.0
415          3.0          3191.0
416          3.0          3191.0
417          3.0          3191.0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 831 entries, 413 to 1270
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MQ category     831 non-null    float64
 1   Zipcode-number  831 non-null    float64
dtypes: float64(2)
memory usage: 19.5 KB
None


In [53]:
fig = px.scatter_mapbox(MQ_Category, lat="latitude", lon="longitude",  color='MQ category', zoom=6,title='MQ category location', height=500)
fig.update_layout(
    title={
        'text': "MQ category distribution by location",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
fig.show()